In [5]:
import os
if os.path.exists("./SPADE"): os.chdir("./SPADE")
    
import pickle
from models.pix2pix_model import Pix2PixModel
from options.test_options import TestOptions
import numpy as np

class MyOptions(TestOptions):
    def initialize(self, parser):
        super().initialize(parser)
        parser.set_defaults(name="coco_pretrained", dataset_mode="coco", dataroot="./datasets/coco_stuff")
        return parser

    def gather_options(self):
        import data
        import models
        import argparse
        # initialize parser with basic options
        if not self.initialized:
            parser = argparse.ArgumentParser(
                formatter_class=argparse.ArgumentDefaultsHelpFormatter)
            parser = self.initialize(parser)

        # get the basic options
        opt, unknown = parser.parse_known_args()

        # modify model-related parser options
        model_name = opt.model
        model_option_setter = models.get_option_setter(model_name)
        parser = model_option_setter(parser, self.isTrain)

        # modify dataset-related parser options
        dataset_mode = opt.dataset_mode
        dataset_option_setter = data.get_option_setter(dataset_mode)
        parser = dataset_option_setter(parser, self.isTrain)

        opt, unknown = parser.parse_known_args()

        # if there is opt_file, load it.
        # The previous default options will be overwritten
        if opt.load_from_opt_file:
            parser = self.update_options_from_file(parser, opt)

        opt = parser.parse_args(args=[])
        self.parser = parser
        return opt

opt = MyOptions().parse()
model = Pix2PixModel(opt)
model.eval()

N=182
import base64
from PIL import Image
from io import BytesIO
import torch
import util.util
c = util.util.Colorize(N)
c = c.cmap.numpy()

def generate(src):
    arr = np.array(Image.open(BytesIO(base64.b64decode(src.split(",")[-1]))).resize((256,256)))[:,:,:3]
    globals()["arr"] = 1
    arr2 = np.linalg.norm((c.reshape(N,1,1,3).repeat(256,axis=1).repeat(256,axis=2) - arr),axis=3).argmin(axis=0).reshape(1,1,256,256)
    arr3 = torch.cuda.FloatTensor(1,184,256,256).zero_().scatter_(1,torch.from_numpy(arr2).cuda(),1.0)
    arr4 = util.util.tensor2im(model.netG(arr3)).reshape(256,256,3)
    globals()["arr4"] = arr4
    byte_io = BytesIO()
    Image.fromarray(arr4).save(byte_io, 'PNG')
    dst = b'data:image/png;base64,'+base64.b64encode(byte_io.getvalue())
    return dst

from IPython.display import HTML
HTML("../main.html")
